# Trabalho de Implementação - Heurísticas e Metaheurísticas
## Thiago Pádua de Carvalho - 2020007066

### Heurística Construtiva - Nearest Neighbour
A heurística construtiva escolhida foi a Nearest Neighbour. Sua estratégia é simples e consegue atingir uma boa aproximação do valor ótimo de maneira relativamente eficiente. A ideia é começar em um vértice arbitrário e, a partir dele, escolher o mais próximo que ainda não foi visitado. O algoritmo termina quando um ciclo hamiltoniano com nós correspondentes a todas as cidades é completo. A complexidade é O(n²) no pior caso, onde n corresponde ao número de cidades (pontos).

#### Implementação
Instâncias geométricas, onde as cidades correspondem a pontos no plano e a distância entre duas cidades depende de suas coordenadas apresentam a vantagem de possibilitar descartar rapidamente grandes grupos de arestas, aproveitando a geometria com estruturas de dados apropriadas.

##### **K-D Tree**
Em particular, para pontos no plano, é possível construir em tempo O(Nlog⁡N) uma estrutura de dados (ED) que permita responder a consultas de vizinhos mais próximos em tempo médio bem inferior a O(N) por consulta. A estrutura escolhida foi a k-d tree, com k = 2 dimensões.

Árvores desse tipo são similares a Binary Seach Trees. A diferença é que, em vez de comparar a chave de busca com a chave do nó atual, ela é comparada com a coordenada correspondente do nó atual. A cada nível da árvore, a dimensão da comparação é alternada. A árvore é construída recursivamente, de maneira que cada nó acrescentado divide o espaço em duas regiões menores, até que cada nó contenha um ponto. Desse modo, o número de operações necessárias para encontrar vizinhos mais próximos geralmente é em média O(log⁡N), quando a árvore á balanceada

![K-D Tree](KDtree.png)

Sendo assim, os passos da heurística são os seguintes:
```
1. Construir a k-d tree com as cidades
2. Inicializar o ciclo hamiltoniano com uma cidade arbitrária
3. Enquanto houver cidades não visitadas:
    4. Encontrar a cidade mais próxima da última cidade visitada
    5. Adicionar a cidade ao ciclo
    6. Remover a última cidade visitada da k-d tree
7. Adicionar a primeira cidade ao ciclo
```



### Análise dos Resultados

In [5]:
import os
import pandas as pd
import subprocess
import time

In [11]:
df = pd.DataFrame(columns=["Instance", "Heuristic", "Time (s)", "Result"])

# Adiciona uma nova linha usando loc
# df.loc[len(df)] = ["kroA100", "Nearest Neighbor", 0.0023, 12345, None]

directory = "instances/EUC_2D"

# Execute the heuristic for each file in the directory
for file in os.listdir(directory):
    filepath = os.path.join(directory, file)
    start_time = time.time()
    result = subprocess.run(['./heuristic', filepath], capture_output=True, text=True)
    end_time = time.time()
    elapsed_time = end_time - start_time

    # Add the result to the dataframe
    df.loc[len(df)] = [file, "Nearest Neighbor", elapsed_time, result.stdout.strip()]

start_time = time.time()
result = subprocess.run(['./heuristic', "instances/att48.tsp"], capture_output=True, text=True)
end_time = time.time()
elapsed_time = end_time - start_time

df.loc[len(df)] = ["instances/att48.tsp", "Nearest Neighbor", elapsed_time, result.stdout.strip()]

display(df)

,Instance,Heuristic,Time (s),Result
0,rat99.tsp,Nearest Neighbor,0.029607,1752.61
1,kroB150.tsp,Nearest Neighbor,0.019424,41567.4
2,kroA150.tsp,Nearest Neighbor,0.011775,44618.1
3,pr152.tsp,Nearest Neighbor,0.012801,113798
4,rat195.tsp,Nearest Neighbor,0.015322,3737.81
5,lin105.tsp,Nearest Neighbor,0.012258,26881.9
6,st70.tsp,Nearest Neighbor,0.008688,919.471
7,berlin52.tsp,Nearest Neighbor,0.008643,8965.52
8,pr76.tsp,Nearest Neighbor,0.010270,159624
9,kroA200.tsp,Nearest Neighbor,0.009790,46895.9
